In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers
import pickle

2023-03-12 10:10:55.244213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
aggregated_data = pickle.load( open( '../data/processed/20230310_dataAugmentation/aggregated_data', 'rb' ) )

In [3]:
[train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca,
    train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca,
    train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca,
    train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca,
    train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca,
    train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca,
    train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca,
    train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca,
    train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca,
    train_vgg_raw_pca, valid_vgg_raw_pca, test_vgg_raw_pca,
    train_vgg_bg_pca, valid_vgg_bg_pca, test_vgg_bg_pca,
    train_vgg_fg_pca, valid_vgg_fg_pca, test_vgg_fg_pca,
    train_panns_clip_raw_pca, valid_panns_clip_raw_pca, test_panns_clip_raw_pca,
    train_panns_clip_bg_pca, valid_panns_clip_bg_pca, test_panns_clip_bg_pca,
    train_panns_clip_fg_pca, valid_panns_clip_fg_pca, test_panns_clip_fg_pca,
    train_panns_embedding_raw_pca, valid_panns_embedding_raw_pca, test_panns_embedding_raw_pca,
    train_panns_embedding_bg_pca, valid_panns_embedding_bg_pca, test_panns_embedding_bg_pca,
    train_panns_embedding_fg_pca, valid_panns_embedding_fg_pca, test_panns_embedding_fg_pca,
    y_train, y_valid, y_test] = aggregated_data

In [4]:
# including YAMNet
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_embedding_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_embedding_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_embedding_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_embedding_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_embedding_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_embedding_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_embedding_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_embedding_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_embedding_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [5]:
# # excluding YAMNet
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_vgg_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_vgg_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_vgg_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_vgg_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_vgg_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_vgg_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_vgg_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_vgg_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_vgg_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [6]:
# # including panns embeddings
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_panns_embedding_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_panns_embedding_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_panns_embedding_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_panns_embedding_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_panns_embedding_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_panns_embedding_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_panns_embedding_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_panns_embedding_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_panns_embedding_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [7]:
# as the first half of the loaded data is the original and the second half is the augmented, they need to be permuted
import random
index_permute = random.sample(range(train_data_raw.shape[0]), train_data_raw.shape[0])

In [8]:
train_data_raw = train_data_raw[index_permute]
train_data_bg = train_data_bg[index_permute]
train_data_fg = train_data_fg[index_permute]
train_data_all = train_data_all[index_permute]
y_train = y_train.iloc[index_permute]

# Lazy predict

In [9]:
# from lazypredict.Supervised import LazyClassifier
# clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_vgg_bg_pca, valid_vgg_bg_pca, y_train, y_valid)
# models

# Tensorflow parameter tuning

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt


In [11]:
def build_model_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=200, step=10),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model



In [12]:
def build_model_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=500, step=50),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [13]:
tuner_raw = kt.BayesianOptimization(build_model_narrow,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model/data_augmentation',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_raw_0310_YAMNet')

tuner_bg = kt.BayesianOptimization(build_model_narrow,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model/data_augmentation',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_bg_0310_YAMNet')

tuner_fg = kt.BayesianOptimization(build_model_narrow,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model/data_augmentation',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_fg_0310_YAMNet')

tuner_all = kt.BayesianOptimization(build_model_wide,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model/data_augmentation',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_all_0310_YAMNet')

INFO:tensorflow:Reloading Tuner from ../model/data_augmentation/kt_Bayesian_raw_0310_YAMNet/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/data_augmentation/kt_Bayesian_bg_0310_YAMNet/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/data_augmentation/kt_Bayesian_fg_0310_YAMNet/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/data_augmentation/kt_Bayesian_all_0310_YAMNet/tuner0.json


In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [15]:
tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)

INFO:tensorflow:Oracle triggered exit


In [16]:
tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

INFO:tensorflow:Oracle triggered exit


In [17]:
tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)

INFO:tensorflow:Oracle triggered exit


In [18]:
tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

INFO:tensorflow:Oracle triggered exit


In [19]:
tuner_raw.results_summary(1)

Results summary
Results in ../model/data_augmentation/kt_Bayesian_raw_0310_YAMNet
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.4971176991977589
num_layers: 1
units_0: 60
activation: relu
lr: 0.0001
units_1: 200
units_2: 200
units_3: 10
units_4: 200
units_5: 200
units_6: 200
units_7: 10
units_8: 200
units_9: 190
Score: 0.6955063939094543


In [20]:
tuner_bg.results_summary(1)

Results summary
Results in ../model/data_augmentation/kt_Bayesian_bg_0310_YAMNet
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.33758201281611844
num_layers: 4
units_0: 130
activation: relu
lr: 0.0006562536901904111
units_1: 60
units_2: 180
units_3: 80
units_4: 110
units_5: 20
units_6: 100
units_7: 50
Score: 0.6948356628417969


In [21]:
tuner_fg.results_summary(1)

Results summary
Results in ../model/data_augmentation/kt_Bayesian_fg_0310_YAMNet
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 10
activation: relu
lr: 0.0001
units_1: 10
units_2: 200
units_3: 200
units_4: 200
units_5: 200
units_6: 200
units_7: 200
units_8: 200
units_9: 10
Score: 0.6623072028160095


In [22]:
tuner_all.results_summary(1)

Results summary
Results in ../model/data_augmentation/kt_Bayesian_all_0310_YAMNet
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.5
num_layers: 1
units_0: 460
activation: relu
lr: 0.0001
units_1: 460
units_2: 10
units_3: 10
units_4: 460
units_5: 10
units_6: 460
units_7: 10
units_8: 460
units_9: 10
Score: 0.7015426158905029
